In [ ]:
# Reduce
RDD.reduce(<func>)
사용자가 지정하는 함수를 받아 여러 개의 값을 하나로 줄여줍니다.

In [ ]:
from operator import add

In [ ]:
sample_rdd = sc.parallelize([1, 2, 3, 4, 5])
sample_rdd

In [ ]:
#RDD에 있는 숫자의 합계를 계산:
sample_rdd.reduce(add)

In [ ]:
sample_rdd = sc.parallelize([1, 2, 3, 4])
sample_rdd.reduce(lambda x, y : (x * 2)+ y)

In [ ]:
sc.parallelize([1, 2, 3, 4], 2).reduce(lambda x, y : (x * 2) + y)

In [ ]:
# 파티션을 3개로 지정

In [ ]:
# Fold
RDD.fold(zeroValue, <func>)
reduce와 비슷하지만, zeroValue에 넣어놓고 싶은 시작값을 지정해서 reduce가 가능
zeroValue는 파티션 마다 계산이 일어날 때 하나씩 더해지는 값
fold()를 사용하여 합계를 구하되, 초기값을 지정

In [ ]:
rdd = sc.parallelize([2, 3, 4], 4)

#숫자의 곱을 계산해서 합한다
print(rdd.reduce(lambda x, y : (x * y))) # 2 x 3 -> x 4
print(rdd.fold(1, lambda x , y : (x * y))) # 1 x 2 x 3 x 4

In [ ]:
rdd.fold(2, lambda x , y : (x * y)) 
# 초기값 2 지정 :: 2x2 , 2x3, 2x4, "" > 4 , 6, 8, "" 
# 다시 초기값 2 지정 :: 4x2*6, 48x2*8, "" > 768

In [ ]:
print(rdd.reduce(lambda x, y : x + y)) 
print(rdd.fold(1, lambda x, y : x + y)) 

In [ ]:
# 대규모 숫자 데이터셋 생성
numbers_rdd = sc.parallelize(range(1, 1000001))

# fold를 사용하여 총합계 계산
total_sum = numbers_rdd.fold(0, lambda x, y: x + y)

print(f"Total sum: {total_sum}")

In [ ]:
# 직원 급여 데이터
salaries_rdd = sc.parallelize([50000, 60000, 55000, 75000, 65000, 80000])

# fold를 사용하여 최대 급여 찾기
max_salary = salaries_rdd.fold(0, lambda x, y: max(x, y))

print(f"Maximum salary: {max_salary}")

In [ ]:
# 부서별 직원 급여 데이터
dept_salaries = [("IT", 50000), ("HR", 45000), ("IT", 60000), 
                 ("Finance", 55000), ("HR", 50000), ("IT", 65000)]
dept_rdd = sc.parallelize(dept_salaries)

# foldByKey를 사용하여 부서별 최대 급여 찾기
max_salary_by_dept = dept_rdd.foldByKey(0, lambda x, y: max(x, y))

print("Maximum salary by department:")
for dept, max_salary in max_salary_by_dept.collect():
    print(f"{dept}: {max_salary}")

In [ ]:
# GroupBy
RDD.groupBy(<func>)
그룹핑 함수를 받아 reduction
groupBy() 메서드는 주어진 함수를 기준으로 RDD의 요소들을 그룹화합니다.

In [ ]:
rdd = sc.parallelize([1, 1, 2, 3, 5, 8])
result = rdd.groupBy(lambda x : x % 2).collect()

sorted([(x, sorted(y)) for (x, y) in result])

In [ ]:
[(x, sorted(y)) for (x, y) in result]

In [ ]:
#판매 데이터 분석 (카테고리별 총 매출 계산)
sales_data = [
    ("Electronics", 1000),
    ("Clothing", 500),
    ("Electronics", 1500),
    ("Books", 300),
    ("Clothing", 750),
    ("Books", 200)
]

In [ ]:
# RDD 생성
sales_rdd = sc.parallelize(sales_data)

# 카테고리별 총 매출 계산
category_sales = sales_rdd.groupBy(lambda x: x[0]) \
    .mapValues(lambda values: sum(item[1] for item in values)) \
    .collect()

# 결과 출력
for category, total_sales in category_sales:
    print(f"{category}: ${total_sales}")

In [ ]:
#로그 데이터 처리 (IP 주소별 접속 횟수 분석)
# SparkContext 생성 가정 (sc)
log_data = [
    ("192.168.1.1", "login"),
    ("192.168.1.2", "purchase"),
    ("192.168.1.1", "view"),
    ("192.168.1.3", "login"),
    ("192.168.1.2", "login"),
    ("192.168.1.1", "purchase")
]

# RDD 생성
log_rdd = sc.parallelize(log_data)


In [ ]:
#학생 성적 데이터 분석 (과목별 평균 성적 계산)
# SparkContext 생성 가정 (sc)
student_grades = [
    ("Math", 85),
    ("English", 90),
    ("Math", 92),
    ("Science", 88),
    ("English", 95),
    ("Science", 82)
]

# RDD 생성
grades_rdd = sc.parallelize(student_grades)
